In [1]:
import torch
import torch.nn as nn

import pickle
import numpy as np

loadpath = "processed_data_bert_expand"
bert_data_path = "bert_expand.pkl"

bert-serving-start -model_dir uncased_L-12_H-768_A-12 -num_worker 1  -port 1355 -max_seq_len 150 -device_map 3 -show_tokens_to_client

## Predict Dataset

Read dataset

In [2]:
with open(loadpath, "rb") as f:
    output = pickle.load(f)
clean_data = output["clean_data"]
reduced_data = output["reduced_data"]
token_data = output["token_data"]

Predict dataset

In [3]:
from bert_serving.client import BertClient
bc = BertClient(port=1355)
print("Start predicting")
bert_output = bc.encode(clean_data)

Start predicting


/tmp2/b05505004/intern/lib/python3.7/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=150" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [4]:
bert_data = {
    "clean_data": clean_data,
    "reduced_data": reduced_data,
    "token_data": token_data,
    "bert_data": bert_output
}
with open(bert_data_path, "wb") as f:
    pickle.dump(bert_data, f)

Read dataset with bert sentence representation

In [5]:
with open(bert_data_path, "rb") as f:
    bert_data = pickle.load(f)
clean_data = bert_data["clean_data"]
reduced_data = bert_data["reduced_data"]
token_data = bert_data["token_data"]
bert_output = bert_data["bert_data"]

In [6]:
print("Type: ", type(bert_output), bert_output.shape)
bert_tensor = torch.from_numpy(bert_output)
print(bert_tensor.size())

Type:  <class 'numpy.ndarray'> (100868, 768)
torch.Size([100868, 768])


## Testing

In [66]:
from bert_serving.client import BertClient
bc = BertClient(port=1355)
bert_norm = bert_tensor / torch.norm(bert_tensor, dim=1).view(-1, 1)

def predict(test_sentence, num_related):
    print("Start predicting: {}".format(test_sentence))
    test_array, token = bc.encode([test_sentence], show_tokens=True)
    print(token)
    test_tensor = torch.tensor(test_array[0])
    #print("bert_tensor:", bert_tensor.size()) # torch.Size([100868, 768])
    
    test_norm = test_tensor / torch.norm(test_tensor)
    similarity = torch.matmul(bert_norm, test_norm.view(-1,1))
    
    rank = torch.argsort(similarity, dim=0, descending=True)

    for i in range(1, num_related + 1):
        print("\n=" * 10 + "Similarity: {}".format(similarity[rank[i]][0]) + "=" * 10)
        print(reduced_data[rank[i]])
        print()
        print(clean_data[rank[i]])
    return None

In [67]:
import random

for i, index in enumerate(random.sample(range(len(clean_data)), 10)):
    print("Query: {}".format(reduced_data[index]))
    predict(clean_data[index], 3)
    print("\n" + "*" * 50 + "\n")

Start predicting: Unable to update my PowerDVD 17 Ultra to the latest build.
Start predicting: unable to update my powerdvd 17 ultra to the latest
[['[CLS]', 'unable', 'to', 'update', 'my', 'power', '##d', '##vd', '17', 'ultra', 'to', 'the', 'latest', '[SEP]']]
==========Similarity: tensor([0.9232])==========
Unable to download PowerDVD 18 on my new computer

unable to download powerdvd 18 on my new computer
==========Similarity: tensor([0.9217])==========
Unable to access the PH10 365 on my mac computer.

unable to access the ph10 365 on my mac
==========Similarity: tensor([0.9201])==========
I am unable to upgrade to PowerDVD 18 Ultra which I recently purchased.

i am unable to upgrade to powerdvd 18 ultra which i recently

**************************************************

Start predicting: Hi there, I reboot my pc a lot and my licence says it has been used too many times. It is just myself using the code. Could you please reset GM399-QR9DU-2W768-KXYUE-BWYXT-3HVX8    thank you
Star

[['[CLS]', 'hi', 'there', ',', 'i', "'", 've', 'been', 'away', 'for', 'a', 'few', 'weeks', 'without', 'access', 'to', 'check', 'emails', ',', 'could', 'you', 'be', 'so', 'kind', 'to', 'resume', 'this', 'investigation', 'to', 'resolve', 'the', 'issue', ',', 'please', '?', 'i', 'have', 'already', 'uploaded', 'the', 'files', 'as', 'previously', 'many', 'thanks', 'in', 'advance', 'for', 'your', '[SEP]']]
==========Similarity: tensor([0.9366])==========
Hi There,  I have had so issues with my computer so would it be possible for you to re send the links to my software so I can reinstall it.  This is for all of my software programs.  Many thanks  Leigh

hi there , i have had so issues with my computer so would it be possible for you to re send the links to my software so i can reinstall this is for all of my software many thanks leigh
==========Similarity: tensor([0.9340])==========
Good Morning,    After purchasing the product last night I have had trouble downloading and installing the app

[['[CLS]', 'please', 'fix', 'the', 'profile', 'having', 'to', 'go', 'through', 'every', 'month', 'and', 'year', 'to', 'get', 'to', 'the', 'tight', 'year', 'is', 'ob', '##no', '##xious', '!', '!', 'you', 'should', 'be', 'able', 'to', 'just', 'type', 'the', 'info', 'the', 'options', 'for', 'hair', 'i', 'have', 'a', 'shaved', 'head', ',', 'and', 'there', "'", 's', 'no', 'option', 'for', 'that', ',', 'so', 'to', 'complete', 'and', 'accept', 'my', 'profile', 'info', 'i', 'had', 'to', 'lie', 'about', 'hair', ',', 'not', 'cool', '!', '!', 'attach', 'file', ':', '[SEP]']]
==========Similarity: tensor([0.9422])==========
Can I get a refund??  You guys need to put a HUGE banner on your page to say your software only runs on Windows!! This factoid appears only to be buried in the depths of the spec sheet. I bought a months subscription to see if I like the video editing suite. I run a mac so obviously your software wont run on my system. Please can you refund me my 20 bucks please?! Invoice attac